In [ ]:
import pandas as pd
import sys
import plotly.express as px

from datastory import DataStory

import importlib
sys.path.append('../lib')
import utils, pesys_utils, pandas_utils
importlib.reload(utils)
importlib.reload(pesys_utils)
importlib.reload(pandas_utils)

In [ ]:
utils.set_secrets_as_env()

In [ ]:
con = pesys_utils.open_pen_connection()
df_bpen002 = pandas_utils.pandas_from_sql(sqlfile='../sql/bpen002_oppg.sql', con=con, tuning=10000, lowercase=True)
con.close()

In [ ]:
df_bpen002.head()

In [ ]:
px.bar(df_bpen002, "dato", "antall", color="sakstype")

In [ ]:
df = df_bpen002.copy()
df["dato"] = pd.to_datetime(df.dato)
df.set_index('dato',inplace=True)

df["oppgavetekst_lang"] = df.oppgavetekst.apply(lambda x: pesys_utils.oppgavetekst_lang[x])

In [ ]:
#sakstyper = df.sakstype.unique()
sakstyper = ['Alderspensjon', "Uføretrygd"]
figs = {}
for i in range(len(sakstyper)):
    ytelse = sakstyper[i]
    df_ytelse = df.query(f"sakstype=='{ytelse}'")
    df_mnd = df_ytelse.groupby(["oppgavetekst", "oppgavetekst_lang"])[["antall"]].resample('MS').sum().reset_index()
    figs[ytelse] = px.bar(df_mnd, "dato", "antall", color="oppgavetekst", title=ytelse, hover_data=["oppgavetekst_lang"])

In [ ]:
n_days = 15
for i in range(len(sakstyper)):
    ytelse = sakstyper[i]
    df_now = df[df.index >= df.index.max() - pd.Timedelta(f"{n_days + 1} days")]
    df_ytelse = df_now.query(f"sakstype=='{ytelse}'")
    df_mnd = df_ytelse.groupby(["oppgavetekst", "oppgavetekst_lang", "prioritet"])[["antall"]].sum().reset_index().sort_values("antall", ascending=False)
    figs[ytelse + "_now"] = px.bar(df_mnd, "oppgavetekst", "antall", title=ytelse + f" siste {n_days} dager", color="prioritet", hover_data=["oppgavetekst_lang"], color_discrete_map={'HOY':utils.nav_colors[1], 'LAV':utils.nav_colors[0]})

In [ ]:
for i in range(len(sakstyper)):
    ytelse = sakstyper[i]
    df_ytelse = df.query(f"sakstype=='{ytelse}'")
    df_mnd = df_ytelse.groupby(["prioritet"])[["antall"]].resample('MS').sum().reset_index()
    figs[ytelse + "_prio"] = px.bar(df_mnd, "dato", "antall", color="prioritet", barmode='group', title=ytelse + f" - oppgaveprioritet", color_discrete_map={'HOY':utils.nav_colors[1], 'LAV':utils.nav_colors[0]})

In [ ]:
for _, fig in figs.items():
    fig.show()

In [ ]:
story = DataStory("Oppgaver oppretta av BPEN002")
for _, fig in figs.items():
    story.plotly(fig.to_json())
story.publish(url="https://nada.intern.nav.no/api")